In [1]:
pip install -q yahoo_fin

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.6 MB/s eta 0:00:00


In [2]:
mkdir -p data results log

In [3]:
pip install  git+https://github.com/ahsank/runml#egg=runml

  Cloning https://github.com/ahsank/runml to /tmp/pip-install-1ucydida/runml_89b3b6bbac5b419d9ae10c0bf3957da2
  Running command git clone --filter=blob:none --quiet https://github.com/ahsank/runml /tmp/pip-install-1ucydida/runml_89b3b6bbac5b419d9ae10c0bf3957da2
  Resolved https://github.com/ahsank/runml to commit 322d8f29189102d0fd30767bcf91fd34f2c6e641
  Preparing metadata (setup.py) ... done
  Created wheel for runml: filename=runml-0.0.9-py3-none-any.whl size=10708 sha256=4f8ce031bcc616737655519e27065240b6953c0f6304db820885cefca507c08f
  Stored in directory: /tmp/pip-ephem-wheel-cache-mcy87o3d/wheels/84/b5/4b/fd61fcf9917bbbe2c05e1473601750f58792e4d39e9cac342d
Successfully built runml


In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive
/content/drive/MyDrive/colab/results/ipos-6a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-6a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-6a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-6a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/vols-6b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/vols-6b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-

In [6]:
import numpy as np
import pandas as pd

In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [8]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '9b', tickers, True)

In [12]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
78,WOLF,2.88,0.95,1009.94,948.93,8.10,11.61,0.43,2.19,0.73,1063.04,1128.08,11.24,0.39,1.66,0.78,814.78,988.06,8.55,0.06,9.383721
55,PLUG,1.07,0.90,432.89,454.85,1.94,2.53,0.30,0.76,0.73,464.35,353.54,3.56,0.84,0.58,0.75,178.77,428.65,1.73,-0.11,8.666667
27,FOUR,1.74,0.93,643.19,541.87,86.90,93.54,0.08,0.98,0.82,885.52,713.64,95.92,0.10,0.89,0.63,561.25,484.94,76.55,-0.12,8.425000
64,SNOW,5.08,0.90,1542.56,2486.30,110.27,124.56,0.13,3.76,0.74,1876.85,2645.62,122.62,0.11,2.96,0.75,2496.45,3035.14,112.39,0.02,8.230769
5,BASE,0.66,0.92,168.61,226.44,13.67,14.75,0.08,0.45,0.70,169.12,207.37,15.84,0.16,0.35,0.71,152.37,222.19,12.02,-0.12,8.200000
24,ETSY,3.82,0.90,1360.03,1569.90,53.91,60.24,0.12,2.99,0.71,896.10,1653.78,62.75,0.16,1.99,0.76,1304.36,1688.69,53.47,-0.01,8.166667
37,JKS,1.46,0.90,291.10,596.04,19.74,21.86,0.11,0.93,0.62,408.23,557.43,21.99,0.11,0.58,0.76,521.00,688.75,17.41,-0.12,8.090909
10,CHGG,1.49,0.83,175.53,618.02,1.74,3.08,0.77,1.15,0.63,19.97,324.30,3.17,0.82,1.02,0.69,126.63,576.46,2.25,0.29,8.079221
29,GTLB,1.79,0.90,410.77,466.71,52.05,57.07,0.10,1.06,0.74,491.32,626.96,58.04,0.12,1.00,0.70,447.48,360.47,48.07,-0.08,8.000000
7,BMBL,0.88,0.85,145.98,337.06,6.09,7.10,0.17,0.63,0.70,179.81,284.16,8.21,0.35,0.63,0.79,271.97,393.76,5.39,-0.11,7.617647


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '7b', tickers1, False)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
1,ARKK,RROnly,30.638765,0.952128,501.42,623.64,1125.07,46.63,49.590000,0.06,8.723404
28,VNQ,RROnly,66.185277,0.962766,379.96,267.97,647.93,88.79,84.800003,-0.04,8.696809
46,XNTK,RROnly,90.719074,0.962766,737.07,519.93,1256.99,188.68,195.389999,0.04,8.696809
25,SMOG,RROnly,91.306753,0.941489,569.71,781.26,1350.97,102.35,109.970001,0.07,8.579027
6,DTEC,RROnly,30.712664,0.952128,168.91,163.76,332.67,41.87,43.340000,0.04,8.324468
33,XITK,RROnly,92.325867,0.941489,732.48,974.94,1707.42,148.75,155.460007,0.05,8.244681
22,ONLN,RROnly,26.861564,0.930851,257.59,355.03,612.62,40.50,43.049999,0.06,8.156028
8,FPX,RROnly,77.296899,0.952128,461.09,483.12,944.21,100.17,102.860001,0.03,7.925532
13,IXN,RROnly,40.538678,0.946809,294.73,180.44,475.17,81.24,83.660004,0.03,7.695035
11,IPO,RROnly,24.468567,0.941489,257.14,282.45,539.60,41.25,42.389999,0.03,7.464539


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
lossfn = findata.Huber()
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DPSI', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombinedVola(tickers3, 'vols-8b', mod, True, loss=lossfn)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
47,OPEN,0.45,0.87,105.70,151.95,2.25,3.14,0.40,0.57,0.57,50.83,81.44,3.11,0.38,0.23,0.71,72.81,116.41,2.17,-0.04,4.792647
7,AUGX,0.16,0.89,31.69,35.18,2.29,2.77,0.21,0.20,0.66,30.47,24.49,3.57,0.56,0.09,0.75,26.63,53.38,1.79,-0.22,4.711485
9,ATOM,0.94,0.84,213.24,190.00,3.02,4.46,0.48,0.92,0.52,55.18,66.28,3.58,0.19,0.37,0.70,140.16,115.43,2.82,-0.07,4.607843
37,LFMD,0.52,0.82,137.96,114.75,5.45,8.08,0.48,0.53,0.56,135.86,60.68,6.79,0.25,0.28,0.71,61.96,106.14,4.41,-0.19,4.448529
65,SOUN,0.26,0.88,35.54,36.75,5.03,5.68,0.13,0.30,0.55,14.37,19.81,5.45,0.08,0.09,0.63,15.94,0.50,4.08,-0.19,4.253394
50,PAYO,0.22,0.86,56.29,49.31,7.03,8.01,0.14,0.21,0.64,12.20,42.56,6.99,-0.01,0.09,0.75,57.44,57.21,5.56,-0.21,4.058824
64,SMWB,0.47,0.86,48.44,79.63,8.70,9.66,0.11,0.41,0.58,22.68,74.37,9.09,0.05,0.19,0.81,71.96,133.52,8.02,-0.08,3.786096
57,RUM,0.31,0.81,50.42,87.15,5.93,6.92,0.17,0.38,0.59,85.42,81.18,7.63,0.29,0.13,0.74,80.10,105.69,5.62,-0.05,3.647059
45,OMQS,0.29,0.84,51.38,66.80,0.18,0.20,0.10,0.31,0.55,59.50,59.62,0.23,0.28,0.13,0.80,61.78,71.79,0.13,-0.27,3.341176
36,LAZR,0.58,0.84,149.22,145.94,1.00,1.07,0.07,1.00,0.52,124.89,127.87,1.56,0.56,0.37,0.73,108.04,158.24,1.16,0.16,2.655462


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombinedVola(tickers4, 'mcap-8b', mod4, False)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
1,ACN,3.46,0.96,1857.75,1354.43,341.95,301.59,-0.12,2.27,0.81,2927.17,2819.62,340.99,-0.00,1.97,0.66,1122.41,1125.49,291.10,-0.15,6.523810
36,TSLA,6.50,0.93,3212.94,2900.27,214.11,154.59,-0.28,4.10,0.81,3603.21,3269.94,218.58,0.02,2.96,0.68,2144.59,2346.09,178.17,-0.17,6.392857
37,TXN,1.97,0.96,887.69,649.09,214.34,197.56,-0.08,0.87,0.82,2061.74,1969.87,211.56,-0.01,0.76,0.46,-8.74,362.01,194.34,-0.09,6.357143
4,AMZN,2.35,0.95,1059.40,813.15,178.50,158.48,-0.11,1.53,0.76,969.65,687.96,182.71,0.02,1.12,0.70,871.90,1035.60,159.33,-0.11,6.331169
19,FDX,3.58,0.94,1239.73,1533.26,298.77,258.22,-0.14,2.24,0.78,2319.03,2265.24,298.88,0.00,1.95,0.61,643.05,1010.36,270.08,-0.10,6.285714
2,ADBE,7.48,0.95,3405.37,3438.40,574.41,529.53,-0.08,4.64,0.75,2800.32,3181.36,646.04,0.12,3.90,0.77,3576.11,3939.26,549.75,-0.04,6.160714
20,HD,3.39,0.94,1864.44,1263.22,368.50,333.98,-0.09,1.78,0.79,3225.43,2931.10,384.22,0.04,1.94,0.42,-141.94,826.76,339.45,-0.08,6.047619
28,NKE,1.84,0.92,603.78,940.76,83.32,95.70,0.15,1.18,0.71,816.66,963.13,85.79,0.03,0.93,0.66,327.67,619.49,77.49,-0.07,6.038095
35,TMO,5.72,0.94,2813.38,2550.53,615.07,566.45,-0.08,3.29,0.73,3310.99,2457.18,629.48,0.02,2.29,0.71,2422.89,2220.56,591.22,-0.04,5.964286
10,CDNS,4.12,0.93,1477.23,807.32,268.93,241.66,-0.10,2.25,0.86,1527.25,947.40,289.65,0.08,1.74,0.67,1430.60,781.74,242.57,-0.10,5.942857


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-6b', mod5, True)[0]


In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
61,TKAMY,0.19,0.93,71.73,77.30,3.62,3.72,0.03,0.14,0.78,103.87,128.25,4.03,0.11,0.12,0.72,33.32,79.59,3.62,0.00,1.97619
9,COVTY,0.40,0.91,143.96,148.20,30.61,31.14,0.02,0.39,0.83,303.36,311.46,33.30,0.09,0.28,0.48,25.84,39.62,30.39,-0.01,0.00000
10,CDMGF,0.68,0.73,207.11,320.03,23.11,23.40,0.01,0.98,0.74,1927.18,2078.74,25.40,0.10,0.82,0.03,-926.77,30.15,23.38,0.01,0.00000
33,MGA,1.01,0.93,340.98,506.97,42.03,42.35,0.01,0.88,0.80,1041.36,1016.96,45.00,0.07,0.69,0.59,48.71,343.50,42.08,0.00,0.00000
15,EDNMF,0.31,0.43,125.15,149.55,40.25,42.39,0.05,0.32,0.76,363.25,440.54,42.35,0.05,0.27,0.10,-249.54,77.02,40.92,0.02,0.00000
45,POAHF,1.57,0.92,410.56,610.01,43.29,43.95,0.02,1.10,0.78,578.33,663.12,47.32,0.09,0.86,0.74,497.04,560.85,42.67,-0.01,0.00000
54,SE,4.21,0.90,1571.28,1876.93,78.31,80.94,0.03,4.39,0.66,772.15,1591.70,79.90,0.02,2.60,0.72,1193.03,1795.70,73.82,-0.06,0.00000
57,TEF,0.05,0.91,21.08,18.99,4.52,4.59,0.02,0.04,0.83,97.84,92.08,4.51,-0.00,0.02,0.17,-30.52,2.04,4.47,-0.01,0.00000
66,YARIY,0.27,0.92,118.90,86.54,14.49,14.79,0.02,0.20,0.74,512.99,517.29,15.92,0.10,0.18,0.31,-196.87,53.15,14.04,-0.03,0.00000


In [ ]:
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '8a', tickers6, False)

In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
7,EWG,0.23,0.96,102.28,95.77,31.68,28.12,-0.11,0.15,0.72,70.66,76.11,31.47,-0.01,0.19,0.67,102.58,74.70,28.95,-0.09,9.236364
12,EWP,0.21,0.96,142.77,78.71,33.21,30.74,-0.07,0.16,0.73,108.31,77.59,32.78,-0.01,0.15,0.79,137.15,108.55,29.41,-0.11,9.028571
21,THD,0.43,0.94,225.30,260.92,65.65,58.60,-0.11,0.27,0.66,145.08,183.42,63.07,-0.04,0.28,0.76,224.21,296.94,60.18,-0.08,8.854545
0,ARGT,0.69,0.95,279.10,223.94,64.72,60.38,-0.07,0.46,0.76,263.06,139.34,65.26,0.01,0.41,0.73,281.60,167.41,57.26,-0.12,8.785714
15,EWY,0.71,0.95,248.80,318.21,60.50,64.60,0.07,0.42,0.74,273.58,236.23,64.33,0.06,0.45,0.73,204.37,299.54,57.87,-0.04,8.785714
4,ECH,0.26,0.97,165.88,147.75,25.00,24.32,-0.03,0.15,0.78,166.05,139.70,25.77,0.03,0.14,0.66,129.32,107.05,24.53,-0.02,8.700000
23,VNM,0.14,0.95,65.04,81.25,12.57,13.37,0.06,0.11,0.70,47.36,48.64,12.78,0.02,0.11,0.76,56.08,77.21,12.29,-0.02,8.666667
10,EWC,0.26,0.94,161.97,82.48,39.31,36.27,-0.08,0.16,0.83,152.80,103.87,39.06,-0.01,0.19,0.68,136.23,92.27,35.09,-0.11,8.650000
5,EPHE,0.18,0.95,85.65,120.59,27.58,28.86,0.05,0.10,0.80,109.76,102.24,28.38,0.03,0.14,0.73,79.28,99.71,24.64,-0.11,8.500000
6,EWA,0.18,0.93,111.47,59.68,25.12,23.11,-0.08,0.11,0.84,133.07,66.03,24.91,-0.01,0.11,0.64,91.40,37.53,23.29,-0.07,8.425000


In [ ]:
findata.EPOCHS=200
review = [ 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
df7 = run_model('val', '9a', tickers7, True)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
2,ALB,4.76,0.96,2196.83,2421.23,76.90,85.28,0.11,3.17,0.79,2184.78,2524.58,97.33,0.27,2.25,0.75,1785.61,2370.41,79.63,0.04,9.236364
37,FCX,0.80,0.97,352.36,295.53,40.00,42.37,0.06,0.48,0.78,438.95,294.27,44.88,0.12,0.39,0.76,427.02,333.85,37.27,-0.07,9.200000
12,BGFV,0.40,0.92,136.50,151.87,2.12,3.13,0.48,0.38,0.70,274.10,250.96,2.51,0.18,0.23,0.74,120.76,181.93,1.68,-0.21,9.033333
0,AAL,0.34,0.93,170.91,129.51,10.80,13.48,0.25,0.18,0.76,190.35,182.11,12.85,0.19,0.17,0.80,159.30,165.57,10.48,-0.03,9.020000
73,RH,8.82,0.91,3008.27,4220.16,248.09,330.04,0.33,5.44,0.66,2244.51,3609.01,305.48,0.23,4.51,0.72,2368.56,2964.78,241.69,-0.03,8.827273
33,DVN,0.94,0.93,428.39,307.30,41.34,47.04,0.14,0.62,0.80,363.72,225.09,43.32,0.05,0.59,0.65,376.52,175.96,39.67,-0.04,8.800000
16,CC,0.57,0.91,259.27,262.64,17.47,21.18,0.21,0.30,0.79,299.60,352.91,21.96,0.26,0.33,0.71,250.22,255.04,16.58,-0.05,8.671429
67,PETS,0.51,0.89,80.74,173.38,3.29,4.32,0.31,0.38,0.61,203.31,271.22,3.93,0.20,0.25,0.87,171.96,244.01,3.13,-0.05,8.545161
28,COP,1.51,0.93,771.70,340.65,106.02,113.74,0.07,1.09,0.76,748.54,335.09,116.73,0.10,0.90,0.73,701.58,449.97,98.38,-0.07,8.300000
30,DAL,0.60,0.94,344.47,282.16,42.03,43.92,0.05,0.35,0.72,281.31,312.72,42.72,0.02,0.29,0.73,279.29,306.18,38.88,-0.07,8.200000


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXB', 'FXE', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '9a', tickers8, True)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
9,TLT,1.21,0.96,261.01,449.66,100.41,94.09,-0.06,1.16,0.65,65.41,273.97,103.12,0.03,0.85,0.82,211.38,460.47,99.47,-0.01,8.933333
6,SLV,0.31,0.96,109.71,107.74,28.02,26.97,-0.04,0.26,0.70,113.63,103.20,27.95,-0.00,0.17,0.79,138.19,148.88,25.72,-0.08,8.600000
7,SOYB,0.30,0.93,102.31,76.26,22.12,21.05,-0.05,0.24,0.66,71.13,50.40,22.16,0.00,0.24,0.76,117.17,103.71,20.10,-0.09,7.900000
3,GLD,1.40,0.93,633.32,436.18,238.68,229.37,-0.04,1.44,0.75,633.73,328.44,237.72,-0.00,0.92,0.63,352.18,80.26,219.89,-0.08,7.550000
13,WEAT,0.14,0.88,37.19,42.60,5.30,5.74,0.08,0.12,0.64,25.33,38.61,5.51,0.04,0.08,0.77,40.95,54.25,5.07,-0.04,7.300000
10,USO,1.30,0.93,461.72,273.47,69.84,71.08,0.02,1.14,0.66,272.74,169.49,73.89,0.06,0.89,0.65,397.22,244.27,67.95,-0.03,5.800000
2,FXY,0.55,0.90,99.28,195.93,65.60,63.70,-0.03,0.59,0.56,30.59,115.29,67.00,0.02,0.45,0.82,55.07,183.49,63.87,-0.03,5.666667
4,ISHG,0.36,0.89,81.00,141.94,73.42,74.88,0.02,0.31,0.69,64.59,89.53,74.17,0.01,0.34,0.77,63.42,114.14,72.36,-0.01,3.400000
1,FXE,0.40,0.93,150.27,135.34,102.28,103.29,0.01,0.46,0.67,103.73,119.98,103.95,0.02,0.34,0.77,127.11,170.51,102.45,0.00,2.300000
5,PDBC,0.17,0.91,58.59,31.67,13.05,13.15,0.01,0.18,0.62,29.21,18.25,13.49,0.03,0.10,0.67,53.52,37.39,13.02,-0.00,0.100000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '9a', tickers9, False)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
11,MATIC-USD,0.05,0.88,7.13,12.81,0.42,0.56,0.33,0.03,0.67,8.82,13.35,1.02,1.41,0.01,0.73,8.93,11.70,0.37,-0.13,5.922078
8,ICP-USD,0.46,0.89,117.01,163.30,8.65,7.68,-0.11,0.41,0.65,140.78,175.97,9.19,0.06,0.19,0.73,112.44,179.51,7.52,-0.13,5.357143
14,TON-USD,0.10,0.88,19.87,30.47,1.00,1.13,0.12,0.11,0.70,59.28,72.70,1.31,0.30,0.03,0.77,24.77,43.49,0.91,-0.10,5.285714
6,ETH-USD,77.31,0.87,22744.09,23195.91,2429.74,2079.19,-0.14,59.94,0.67,13314.51,19795.80,2530.01,0.04,28.64,0.69,17327.94,12427.05,2102.62,-0.13,5.285714
16,XRP-USD,0.02,0.89,5.23,6.04,0.58,0.52,-0.10,0.01,0.66,4.99,5.42,0.58,0.00,0.00,0.79,5.62,5.97,0.44,-0.24,5.257143
15,TRX-USD,0.00,0.87,0.89,0.35,0.15,0.13,-0.13,0.00,0.78,0.99,0.53,0.15,0.02,0.00,0.56,0.52,0.32,0.13,-0.15,5.214286
4,DOGE-USD,0.00,0.85,0.88,1.40,0.11,0.09,-0.12,0.00,0.55,0.73,1.37,0.11,0.06,0.00,0.73,0.75,1.32,0.09,-0.16,4.821429
7,FIL-USD,0.51,0.83,90.37,147.48,3.71,3.27,-0.12,0.41,0.70,95.33,132.67,3.95,0.06,0.19,0.71,62.10,115.45,3.05,-0.18,4.511905
5,DOT-USD,0.45,0.85,104.00,159.30,4.42,4.04,-0.09,0.31,0.67,49.19,127.90,4.45,0.01,0.18,0.76,79.09,135.49,3.77,-0.15,4.404762
0,ADA-USD,0.02,0.88,5.86,7.25,0.36,0.34,-0.06,0.02,0.65,5.34,7.12,0.37,0.04,0.01,0.74,4.46,9.49,0.32,-0.10,4.285714


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-9a-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
com-9a-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
com-9a-low-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
cry-9a-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
cry-9a-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
cry-9a-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
cry-9a-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
cry-9a-low